In [6]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random
# ==== 基因演算法會用到的函式 ====    # === Step 5. 設定適應度函數 ===
def initPop():             # 初始化群體
    p = []

    # === 編碼 000111222 的排列  ===
    for i in range(NUM_CHROME) :        
        a = []
        for j in range(NUM_JOB):
            for k in range(NUM_MACHINE):
                a.append(j)
        np.random.shuffle(a)

        p.append(a)
        
    return p

def fitFunc(x):            # 適應度函數
    S = np.zeros((NUM_JOB, NUM_MACHINE))    # S[i][j] = Starting time of job i at machine j
    C = np.zeros((NUM_JOB, NUM_MACHINE))    # C[i][j] = Completion time of job i at machine j
    
    B = np.zeros(NUM_MACHINE, dtype=int)    # B[j] = Available time of machine j
    
    opJob = np.zeros(NUM_JOB, dtype=int)    # opJob[i] = current operation ID of job i
    
    for i in range(NUM_BIT):
        m = mOrder[x[i]][opJob[x[i]]]
        if opJob[x[i]] != 0:
            S[x[i]][m] = max([B[m], C[x[i]][mOrder[x[i]][opJob[x[i]]-1]]])
        else:
            S[x[i]][m] = B[m]
            
        C[x[i]][m] = B[m] = S[x[i]][m] + pTime[x[i]][opJob[x[i]]]
        
        opJob[x[i]] += 1
            
    return -max(B)           # 因為是最小化問題

def evaluatePop(p):        # 評估群體之適應度
    return [fitFunc(p[i]) for i in range(len(p))]

def selection(p, p_fit):   # 用二元競爭式選擇法來挑父母
	a = []

	for i in range(NUM_PARENT):
		[j, k] = np.random.choice(NUM_CHROME, 2, replace=False)  # 任選兩個index
		if p_fit[j] > p_fit[k] :                      # 擇優
			a.append(p[j].copy())
		else:
			a.append(p[k].copy())

	return a

def crossover_one_point(p):           # 用單點交配來繁衍子代 (new)
	a = []

	for i in range(NUM_CROSSOVER) :
		c = np.random.randint(1, NUM_BIT)      		  # 隨機找出單點(不包含0)
		[j, k] = np.random.choice(NUM_PARENT, 2, replace=False)  # 任選兩個index
     
		child1, child2 = p[j].copy(), p[k].copy()
		remain1, remain2 = list(p[j].copy()), list(p[k].copy())     # 存還沒被用掉的城市
       
		for m in range(NUM_BIT):
			if m < c :
				remain2.remove(child1[m])   # 砍掉 remain2 中的值是 child1[m]
				remain1.remove(child2[m])   # 砍掉 remain1 中的值是 child2[m]
		
		t = 0
		for m in range(NUM_BIT):
			if m >= c :
				child1[m] = remain2[t]
				child2[m] = remain1[t]
				t += 1
		
		a.append(child1)
		a.append(child2)

	return a

def crossover_uniform(p):           # 用均勻交配來繁衍子代 (new)
	a = []

	for i in range(NUM_CROSSOVER) :
		mask = np.random.randint(2, size=NUM_BIT)
		[j, k] = np.random.choice(NUM_PARENT, 2, replace=False)  # 任選兩個index
     
		child1, child2 = p[j].copy(), p[k].copy()
		remain1, remain2 = list(p[j].copy()), list(p[k].copy())     # 存還沒被用掉的城市
       
		for m in range(NUM_BIT):
			if mask[m] == 1 :
				remain2.remove(child1[m])   # 砍掉 remain2 中的值是 child1[m]
				remain1.remove(child2[m])   # 砍掉 remain1 中的值是 child2[m]
		
		t = 0
		for m in range(NUM_BIT):
			if mask[m] == 0 :
				child1[m] = remain2[t]
				child2[m] = remain1[t]
				t += 1
		
		a.append(child1)
		a.append(child2)

	return a
def mutation(p):	           # 突變
	for _ in range(NUM_MUTATION) :
		row = np.random.randint(NUM_CROSSOVER_2)  # 任選一個染色體
		[j, k] = np.random.choice(NUM_BIT, 2, replace=False)  # 任選兩個基因
      
		p[row][j], p[row][k] = p[row][k], p[row][j]       # 此染色體的兩基因互換


def sortChrome(a, a_fit):	    # a的根據a_fit由大排到小
    a_index = range(len(a))                         # 產生 0, 1, 2, ..., |a|-1 的 list
    
    a_fit, a_index = zip(*sorted(zip(a_fit,a_index), reverse=True)) # a_index 根據 a_fit 的大小由大到小連動的排序
   
    return [a[i] for i in a_index], a_fit           # 根據 a_index 的次序來回傳 a，並把對應的 fit 回傳

def replace(p, p_fit, a, a_fit):            # 適者生存
    b = np.concatenate((p,a), axis=0)               # 把本代 p 和子代 a 合併成 b
    b_fit = p_fit + a_fit                           # 把上述兩代的 fitness 合併成 b_fit
    
    b, b_fit = sortChrome(b, b_fit)                 # b 和 b_fit 連動的排序
    
    return b[:NUM_CHROME], list(b_fit[:NUM_CHROME]) # 回傳 NUM_CHROME 個為新的一個世代

def readfile(file_name):
    with open('cscmax_20_15_10.txt', 'r') as f:
        file_list = [list(map(lambda x: int(x) ,line.split())) for line in f]
    return file_list


def GA_solve(file_name, Pc, Pm, NUM_CHROME, NUM_ITERATION = 2000):
    #np.random.seed(0)          # 若要每次跑得都不一樣的結果，就把這行註解掉
    file_list = readfile(file_name)           # 工件個數   # === Step 1. 根據文檔設定成N個工件 ===
    NUM_JOB = file_list[0][0]
    NUM_MACHINE = file_list[0][1]                     # === Step 1. 設定文檔設定成M個機台
    
    
    
    # ==== 參數設定(與演算法相關) ====
    NUM_BIT = NUM_JOB * NUM_MACHINE		   # 染色體長度 # === Step 3-1. 編碼是 000111222 的排列 ===
    NUM_PARENT = NUM_CHROME                         # 父母的個數
    NUM_CROSSOVER = int(Pc * NUM_CHROME / 2)        # 交配的次數
    NUM_CROSSOVER_2 = NUM_CROSSOVER*2               # 上數的兩倍
    NUM_MUTATION = int(Pm * NUM_CHROME * NUM_BIT)   # 突變的次數# === Step 3-2. NUM_BIT 要修改成 3 x 3 ===
    
    # === Step 2. 設定 processing time 矩陣(row: job ID; col: 對應其加工機台所用的加工時間) ===設定 job 的機台加工順序 ===
    pTime = [[file_list[i][j] for j in range(len(file_list[i])) if j % 2 == 1] for i in range(1, len(file_list))]
    mOrder = [[file_list[i][j] for j in range(len(file_list[i])) if j % 2 == 0] for i in range(1, len(file_list))]
    
    pop = initPop()             # 初始化 pop
    pop_fit = evaluatePop(pop)  # 算 pop 的 fit

    best_outputs = []                           # 用此變數來紀錄每一個迴圈的最佳解 (new)
    best_outputs.append(np.max(pop_fit))        # 存下初始群體的最佳解

    mean_outputs = []                           # 用此變數來紀錄每一個迴圈的平均解 (new)
    mean_outputs.append(np.average(pop_fit))        # 存下初始群體的最佳解

    for i in range(NUM_ITERATION) :
        parent = selection(pop, pop_fit)            # 挑父母
        offspring = crossover_one_point(parent)     # 單點交配
        mutation(offspring)                         # 突變
        offspring_fit = evaluatePop(offspring)      # 算子代的 fit
        pop, pop_fit = replace(pop, pop_fit, offspring, offspring_fit)    # 取代

        best_outputs.append(np.max(pop_fit))        # 存下這次的最佳解
        mean_outputs.append(np.average(pop_fit))    # 存下這次的平均解

        #print('iteration %d: y = %d'	%(i, -pop_fit[0]))     # fit 改負的
    return best_outputs, mean_outputs, pop[0], -pop_fit[0]

In [ ]:
NUM_ITERATION_GENERAL = 100         # 世代數(迴圈數)
NUM_CHROME_GENERAL = 20             # 染色體個數
NUM_BIT_GENERAL = 4                 # 染色體長度

SIGMA = 0.001                 # 生成子代時用到的干擾
#np.random.seed(0)          # 若要每次跑得都不一樣的結果，就把這行註解掉

# ==== 基因演算法會用到的函式 ====
def initPop_GENERAL():# 初始化群體
    return [[random.uniform(0, 1), random.uniform(0, 1), random.randint(1, 300), 2000] for _ in range(NUM_CHROME_GENERAL)]

def fitFunc_GENERAL(x):            # 適應度函數
    file_name = 'cscmax_20_15_10.txt'
    best_outputs, mean_outputs, sol_x, sol_y = GA_solve(file_name, x[0], x[1], x[2], x[3])
    print(sol_y)
    return sol_y

def evaluatePop_GENERAL(p):        # 評估群體之適應度
    return [fitFunc_GENERAL(p[i]) for i in range(len(p))]

def selection_GENERAL(p):          # 隨機找兩個父母
    [i, j] = np.random.choice(NUM_CHROME_GENERAL, 2, replace=False)  # 任選兩個index
    return [p[i], p[j]]

def reproduction_GENERAL(p):       # 繁衍子代
    return [np.random.uniform(np.min([p[0][j], p[1][j]]), np.max([p[0][j], p[1][j]])) \
            + np.random.uniform(low=-SIGMA, high=SIGMA) for j in range(NUM_BIT_GENERAL)]
    
def replace_GENERAL(p, p_fit, k, k_fit):            # 適者生存
    worstIndex = np.argmin(p_fit)
    p[worstIndex] = k
    p_fit[worstIndex] = k_fit
    
    return p, p_fit

# ==== 主程式 ====
pop_GENERAL = initPop_GENERAL()             # 初始化 pop
pop_fit_GENERAL = evaluatePop_GENERAL(pop_GENERAL)  # 算 pop 的 fit

for i in range(NUM_ITERATION_GENERAL) :
    parent_GENERAL = selection(pop_GENERAL)                     # 挑父母
    kid_GENERAL = reproduction_GENERAL(parent_GENERAL)                  # 生子
    kid_fit_GENERAL = fitFunc_GENERAL(kid_GENERAL)                      # 算子代的 fit
    pop_GENERAL, pop_fit_GENERAL = replace_GENERAL(pop_GENERAL, pop_fit_GENERAL, kid_GENERAL, kid_fit_GENERAL)    # 取代

    bestIndex = np.argmax(pop_fit_GENERAL)              # 找此世代最佳解的索引值
    print('iteration %d: x = %s, y = %f' %(i, pop_GENERAL[bestIndex], pop_fit_GENERAL[bestIndex]))

    

In [7]:
file_name = 'cscmax_20_15_10.txt'
best_outputs, mean_outputs, sol_x, sol_y = GA_solve(file_name, 0.7, 0.1, 2000, 100)
print(sol_x, ":" , sol_y)
# 畫圖
plt.plot(best_outputs)
plt.plot(mean_outputs)
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.show()

NameError: name 'NUM_CHROME' is not defined

In [ ]:
NUM_ITERATION = 2000			# 世代數(迴圈數)

NUM_CHROME = 20				# 染色體個數
NUM_BIT = NUM_JOB * NUM_MACHINE		   # 染色體長度 # === Step 3-1. 編碼是 000111222 的排列 ===

Pc = 0.9    					# 交配率 (代表共執行Pc*NUM_CHROME/2次交配)，交叉率建議設定在 0.6-0.9 之間
Pm = 0.05   					# 突變率 (代表共要執行Pm*NUM_CHROME*NUM_BIT次突變)，變異率可以設定在 0.01-0.1 之間

NUM_PARENT = NUM_CHROME                         # 父母的個數
NUM_CROSSOVER = int(Pc * NUM_CHROME / 2)        # 交配的次數
NUM_CROSSOVER_2 = NUM_CROSSOVER*2               # 上數的兩倍
NUM_MUTATION = int(Pm * NUM_CHROME * NUM_BIT)   # 突變的次數# === Step 3-2. NUM_BIT 要修改成 3 x 3 ===